In [8]:
import glob
import pandas as pd
all_files = glob.glob("data/*.csv")
_dtypes1 = {
    'Last Name': 'string',
    'First Name': 'string',
    'Middle Initial': 'string',
    'UIN': 'string',
    'BDGNBR': 'float64',
    'Access Type': 'string',
    'TOA': 'string',
    'POA': 'string',
    'TOD': 'string',
    'POD': 'string',
    'Appointment Made Date': 'string',
    'Appointment Start Date': 'string',
    'Appointment End Date': 'string',
    'Appointment Cancel Date': 'string',
    'Total People': 'int64',
    'Last Updated By': 'string',
    'POST': 'string',
    'Last Entry Date': 'string',
    'Terminal Suffix': 'string',
    'Visitee Last Name': 'string',
    'Visitee First Name': 'string',
    'Meeting Location': 'string',
    'Meeting Room': 'string',
    'Caller Last Name': 'string',
    'Caller First Name': 'string',
    'CALLER_ROOM': 'string',
    'RELEASEDATE': 'string',
    'Caller Room': 'string',
    'Release Date': 'string',
}


li = [pd.read_csv(filename, dtype=_dtypes1) for filename in all_files]
for _df in li:
    _df.columns = li[0].columns
df = pd.concat(li, axis=0, ignore_index=True)


In [10]:
df.head()

,Last Name,First Name,Middle Initial,UIN,BDGNBR,Access Type,TOA,POA,TOD,POD,...,Last Entry Date,Terminal Suffix,Visitee Last Name,Visitee First Name,Meeting Location,Meeting Room,Caller Last Name,Caller First Name,CALLER_ROOM,RELEASEDATE
0,AABY,KATHERINE,D,U77873,NaN,VA,<NA>,<NA>,<NA>,<NA>,...,9/13/2022 14:10,PS,POTUS,<NA>,WH,SOUTH LAWN,SCHWARTZ,PEYTON,<NA>,12/30/2022
1,AABY,KATHERINE,D,U78064,NaN,VA,<NA>,<NA>,<NA>,<NA>,...,9/12/2022 10:48,JL,POTUS,<NA>,WH,SOUTH LAWN,LEE,JANICE,<NA>,12/30/2022
2,AAGAARD,KIMBERLY,D,U80982,NaN,VA,Sep 19 2022 7:58PM,<NA>,<NA>,<NA>,...,9/18/2022 11:01,TW,Tibbitts,Matthew,WH,WEST WING,WW,TOUR,<NA>,12/30/2022
3,AARONS,ANDREA,C,U79444,NaN,VA,Sep 16 2022 9:22AM,<NA>,<NA>,<NA>,...,9/14/2022 9:06,VW,Office,Visitors,WH,EW TOUR,WAVES,VISITORSOFFICE,<NA>,12/30/2022
4,ABADIER,MINA,M,U78525,NaN,VA,Sep 17 2022 10:50AM,<NA>,<NA>,<NA>,...,9/12/2022 9:40,VW,Office,Visitors,WH,EW TOUR,WAVES,VISITORSOFFICE,<NA>,12/30/2022


In [16]:
from datetime import datetime
timestamp_cols = ["Appointment Made Date", "Appointment Start Date", "Appointment End Date", "Appointment Cancel Date", "Last Entry Date"]
for ts_col in timestamp_cols:
    df[ts_col] = df[~df[ts_col].isna()][ts_col].apply(lambda x: pd.to_datetime(datetime.strptime(x, "%m/%d/%Y %H:%M"), infer_datetime_format=True) if x is not None else None)


In [20]:
def parseTOA(x):
    if x is None:
        return None
    try:
        return pd.to_datetime(datetime.strptime(x, "%b %d %Y %H:%M%p"))
    except:
        return None
df.TOA = df[~df.TOA.isna()].TOA.apply(parseTOA)

In [28]:
df.groupby(["First Name", "Last Name", "Middle Initial"]).size().sort_values(ascending=False).head(10)

First Name  Last Name    Middle Initial
JAMES       LEVINGS      W                 85
MICHAEL     HERZOG       N                 70
DANIEL      KRITENBRINK  J                 60
PROTAZI     SEMINDU      M                 60
KRYSTAL     KAAI         C                 59
RUSSELL     WILSON       A                 58
VIRGINIA    MCQUADE      S                 56
FERN        SATO         E                 56
THEA        KENDLER      D                 51
JENNIFER    ESPARZA      N                 50
dtype: int64

In [29]:
df.groupby(["Visitee First Name", "Visitee Last Name"]).size().sort_values(ascending=False).head(10)

Visitee First Name  Visitee Last Name
Visitors            Office               328644
Ed                  Teleky                10265
Amanda              Trocola                6009
Kevin               Ballen                 4841
Room                1                      3267
Dan                 Via                    2813
Peyton              Schwartz               2622
Gionelly            Mills                  2619
Gianna              Juarez                 2539
GIONELLY            MILLS                  1841
dtype: int64

In [30]:
df.groupby(["First Name", "Middle Initial", "Last Name", "Visitee First Name", "Visitee Last Name"]).size().sort_values(ascending=False).head(10)

First Name  Middle Initial  Last Name  Visitee First Name  Visitee Last Name
FERN        E               SATO       Dan                 Via                  41
PATRICIA    A               DONILON    Jing                Qu                   40
PHILIPPA    C               TARRANT    Amanda              Trocola              39
RUSSELL     A               WILSON     Dan                 Via                  38
ANDREW      N               RABENS     Mike                Pyle                 37
MARCELO     R               VONGLEHN   Amanda              Trocola              36
DAVID       P               MCCABE     Ed                  Teleky               34
OWEN        M               HOSLER     Ed                  Teleky               34
GREGORI     N               MONTEIRO   Amanda              Trocola              33
CARLOS      N               SIGUENZA   Amanda              Trocola              33
dtype: int64

In [54]:
import numpy as np
delta = (df[~df.TOA.isna()].TOA - df[~df.TOA.isna()]["Appointment Start Date"]).dt.seconds / 60
bins = pd.cut(delta, list(np.linspace(-80, +120, 50)))
bins[~bins.isna()].groupby(bins).count()


(-80.0, -75.918]          0
(-75.918, -71.837]        0
(-71.837, -67.755]        0
(-67.755, -63.673]        0
(-63.673, -59.592]        0
(-59.592, -55.51]         0
(-55.51, -51.429]         0
(-51.429, -47.347]        0
(-47.347, -43.265]        0
(-43.265, -39.184]        0
(-39.184, -35.102]        0
(-35.102, -31.02]         0
(-31.02, -26.939]         0
(-26.939, -22.857]        0
(-22.857, -18.776]        0
(-18.776, -14.694]        0
(-14.694, -10.612]        0
(-10.612, -6.531]         0
(-6.531, -2.449]          0
(-2.449, 1.633]       11873
(1.633, 5.714]        28492
(5.714, 9.796]        31214
(9.796, 13.878]       28534
(13.878, 17.959]      24529
(17.959, 22.041]      23417
(22.041, 26.122]      13900
(26.122, 30.204]       9722
(30.204, 34.286]       7047
(34.286, 38.367]       5237
(38.367, 42.449]       3501
(42.449, 46.531]       2979
(46.531, 50.612]       2237
(50.612, 54.694]       1993
(54.694, 58.776]       1412
(58.776, 62.857]        893
(62.857, 66.939]    

In [55]:
duration = (df["Appointment End Date"] - df["Appointment Start Date"]).dt.seconds / 60
bins = pd.cut(duration, list(np.linspace(-80, +120, 50)))
bins[~bins.isna()].groupby(bins).count()


(0.0, 12.121]            6
(12.121, 24.242]         0
(24.242, 36.364]        52
(36.364, 48.485]         3
(48.485, 60.606]        62
                        ..
(1139.394, 1151.515]     0
(1151.515, 1163.636]     0
(1163.636, 1175.758]     8
(1175.758, 1187.879]     0
(1187.879, 1200.0]       4
Length: 99, dtype: int64

In [56]:
people = df["Total People"]
bins = pd.cut(people, list(np.linspace(0, 200, 50)))
bins[~bins.isna()].groupby(bins).count()


Total People
(0.0, 4.082]          197435
(4.082, 8.163]        128359
(8.163, 12.245]        29125
(12.245, 16.327]       18661
(16.327, 20.408]       21142
(20.408, 24.49]         9457
(24.49, 28.571]         8484
(28.571, 32.653]       10617
(32.653, 36.735]        7261
(36.735, 40.816]        8068
(40.816, 44.898]        6606
(44.898, 48.98]         8518
(48.98, 53.061]        14552
(53.061, 57.143]        1766
(57.143, 61.224]        1377
(61.224, 65.306]        2069
(65.306, 69.388]        1450
(69.388, 73.469]        1543
(73.469, 77.551]        1505
(77.551, 81.633]        1184
(81.633, 85.714]        1232
(85.714, 89.796]        1121
(89.796, 93.878]        1259
(93.878, 97.959]        1038
(97.959, 102.041]       2330
(102.041, 106.122]       621
(106.122, 110.204]       216
(110.204, 114.286]       450
(114.286, 118.367]      1037
(118.367, 122.449]       358
(122.449, 126.531]       617
(126.531, 130.612]       629
(130.612, 134.694]       784
(134.694, 138.776]      1102
(